# This notebook was run for r/SGExams and r/teenagers
Change the variable below as needed

In [ ]:
sub = "sgexams"

In [ ]:
import praw
import numpy as np
import math

In [ ]:
poster_activity = pd.read_csv("Datasets/Cleaned"+ sub + "_posters_activity.csv")
commenter_activity = pd.read_csv("Datasets/Cleaned"+ sub + "_commenters_activity.csv")

## This will save you from thousands of RuntimeWarning lines 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Convert the subreddit-topic mapping to a dictionary and assign topics to each row in the Reddit activity datasets

In [ ]:
mapping = subs.set_index('subreddit')['topic'].to_dict()

In [ ]:
poster_activity['topic'] = poster_activity['subreddit']
for key in mapping:
    poster_activity['topic'] = poster_activity['topic'].map(mapping)

commenter_activity['topic'] = commenter_activity['subreddit']
for key in mapping:
    commenter_activity['topic'] = commenter_activity['topic'].map(mapping)

# Topic shifts and subreddit drifts
High level concept from Valensise et al. (2019)

## Mathematical algorithm to quantify shifts between topics  
Repeat the cell below for commenters by replacing posters_activity with commenter_activity in line 1 and Posters with Commenters in the second last line

In [ ]:
for name, segment in posters_activity.groupby('name'): 
    segment = segment.sort_values(by='created_utc')
    encodings = pd.get_dummies(segment.topic)
    result = pd.DataFrame(columns=segment['topic'].unique())
    result = result.sort_index(axis=1)

    bin_size = 10
    x = 0
    df_size = len(encodings)

    for i in range(1,int(df_size/bin_size)+1):
        sum = pd.DataFrame(encodings.iloc[x:i*bin_size].sum()).transpose()
        x+=bin_size
        result = pd.concat([result,sum],axis=0)
        
    angles = []
    for i in range(len(result)-1):
        vector1 = np.array(list(result.iloc[i]))
        vector2 = np.array(list(result.iloc[i+1]))

        dot_product = np.dot(vector1, vector2)
        magnitude1 = np.linalg.norm(vector1)
        magnitude2 = np.linalg.norm(vector2)
        cosine_similarity = dot_product / (magnitude1 * magnitude2)
        
        if cosine_similarity> 1:
            cosine_similarity = 1
        if cosine_similarity<-1:
            cosine_similarity = -1
        
        angle = math.acos(cosine_similarity)
        angle = angle*180/math.pi
        angles.append(round(angle,2))

    angles.append(-1)
    result['angle'] = angles

    ucsv = sub + '/Posters/Topics/' +name +'.csv'
    result.to_csv(ucsv, index  =False)

## Mathematical algorithm to quantify drifts between subreddits within a topic  
Repeat the second cell below for commenters by replacing posters_activity with commenter_activity in line 1 and Posters with Commenters in the fifth last line

In [ ]:
def cosine_similarity(df):
    angles = []
    for i in range(len(df)-1):
        vector1 = np.array(list(df.iloc[i]))
        vector2 = np.array(list(df.iloc[i+1]))

        dot_product = np.dot(vector1, vector2)
        magnitude1 = np.linalg.norm(vector1)
        magnitude2 = np.linalg.norm(vector2)
        cosine_similarity = dot_product / (magnitude1 * magnitude2)

        if cosine_similarity> 1:
            cosine_similarity = 1
        if cosine_similarity<-1:
            cosine_similarity = -1

        angle = math.acos(cosine_similarity)
        angle = angle*180/math.pi
        angles.append(round(angle,2))

    angles.append(-1)
    return angles

In [ ]:
for name, segment in posters_activity.groupby('name'): 
    try:
        segment = segment.sort_values(by='created_utc')
        sub_encode = pd.get_dummies(segment.subreddit)
        top_encode = pd.get_dummies(segment.topic)
        encodings = pd.concat([sub_encode,top_encode],axis=1)
        result = pd.DataFrame(columns=segment['subreddit'].unique())
        result = result.sort_index(axis=1)

        bin_size = 10
        x = 0
        df_size = len(encodings)

        for i in range(1,int(df_size/bin_size)+1):
            sum = pd.DataFrame(encodings.iloc[x:i*bin_size].sum()).transpose()
            x+=bin_size
            result = pd.concat([result,sum],axis=0)

        topics = segment['topic'].unique()

        for t in topics: 
            result[t] = result[t].replace(0.0,np.nan)

        for t in topics:
            set1 = set(top_dict[t])
            set2 = set(segment)
            set3 = set1.intersection(set2)
            tcol = t + ' SimilarAngle'
            result[tcol] = cosine_similarity(result[list(set3)])

        filtered_columns = [col for col in result.columns if 'SimilarAngle' in col]
        angles_df = pd.DataFrame(result[filtered_columns])
        mean_angles = []

        for i in range(len(angles_df)):
            mean_angles.append(round(np.nanmean(angles_df.iloc[i]),2))
        result['Mean Angle'] = mean_angles

        ucsv = sub + '/Posters/Subreddits/' +name +'.csv'
        result.to_csv(ucsv, index=False)
    except:
        failed.append(name)
        continue

# TODO add mathematical reason for failure

In [ ]:
print(failed)